In [225]:
import pandas as pd
from constants import *

weight_dependent = [
"handling",
"intl freight"]

weight_tables = {}

for i in weight_dependent:
    weight_tables[i] = pd.read_csv(f'./CSVs/{i}.csv', dtype = {'weight':'float64'})


customs_value_dependent = ["complementary services",
"keeping"
]

customs_value_tables = {}

for i in customs_value_dependent:
    customs_value_tables[i] = pd.read_csv(f'./CSVs/{i}.csv', dtype = {'customs value':'float64'})


In [226]:
input_data = {'weight':12.0,
    "EXW/FCA value": 123840,
    "currency" : "MXN",
    "height" : 65,
    "length" : 20, 
    "depth" :  10,
    "IGI" : 0.05
}

In [227]:
##Performs currency exchange if needed

if input_data['currency'] != 'MXN':
    input_data['EXW/FCA value'] *= exchange_rates[input_data['currency']]

In [228]:
def calculate_weight():
    global input_data
    height = input_data['height']
    length = input_data['length']
    depth = input_data['depth']
    
    input_weight = input_data['weight']
    vol_weight = (height * length * depth)/5000

    if input_weight > vol_weight:
        return input_weight
    else:
        return vol_weight

In [230]:
def weight_costs_query():
    query_df = pd.DataFrame()
    query_df['weight'] = [weight]

    for table in weight_tables:
        query_df[table] = pd.merge_asof(query_df['weight'],
        weight_tables[table],
        on= 'weight',
        direction = 'forward').iloc[:,1] ##Appends only the second column

    query_df = query_df.to_dict(orient='records') ## It comes as a list of dictionaries
    query_df = query_df[0] ##Gets rid of the list
    query_df.pop('weight')

    ##The intl. freight is in USD, exchange required:
    query_df['intl freight'] *= exchange_rates['USD']

    return query_df

In [231]:
costs = weight_costs_query()

In [232]:
costs

{'handling': 1354, 'intl freight': 2461.2200000000003}

In [233]:
customs_value = input_data['EXW/FCA value'] + costs['intl freight']

In [234]:
def customsval_costs_query():
    query_df = pd.DataFrame()
    query_df['customs value'] = [customs_value]

    for table in customs_value_tables:
        query_df[table] = pd.merge_asof(query_df['customs value'],
        customs_value_tables[table],
        on= 'customs value',
        direction = 'forward').iloc[:,1] ##Appends only the second column

    query_df = query_df.to_dict(orient='records') ## It comes as a list of dictionaries
    query_df = query_df[0] ##Gets rid of the list
    query_df.pop('customs value')

    return query_df   
    

In [235]:
costs.update(customsval_costs_query())

In [236]:
costs['DTA'] = customs_value * 0.008
costs['IGI'] = customs_value * input_data['IGI']

In [237]:
costs.update(fixed_costs)

In [238]:
nonfree_surcharge = costs['additional handling'] + costs['keeping'] + costs['storage']
maneuvers_total = costs['handling'] + costs['maneuvers'] + nonfree_surcharge

In [239]:
def agents_fee():
    values = [customs_value,
    costs['DTA'],
    costs['IGI'],
    278,
    maneuvers_total,
    nonfree_surcharge,
    costs['DHL management surcharge']]
    values_sum = sum(values) * 1.16 * 0.004

    if values_sum > 1000:
        return values_sum
    else:
        return 1000


In [240]:
costs['agents fee'] = agents_fee()

In [241]:
costs

{'handling': 1354,
 'intl freight': 2461.2200000000003,
 'complementary services': 713,
 'keeping': 2808,
 'DTA': 1010.40976,
 'IGI': 6315.061000000001,
 'agent fixed surcharges': 850,
 'additional handling': 656,
 'storage': 604,
 'maneuvers': 322,
 'DHL management surcharge': 448.28,
 'agents fee': 1000}

In [242]:
maneuvers_total

5744